In [3]:
#somehow, the import from fungiclef did not work, so I copied the code from the file
import os
import sys
from contextlib import contextmanager

from pyspark.sql import SparkSession

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable


def get_spark(cores=4, memory="8g", local_dir="/mnt/data/tmp", **kwargs):
    """Get a spark session for a single driver."""
    builder = (
        SparkSession.builder.config("spark.driver.memory", memory)
        .config("spark.driver.cores", cores)
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .config("spark.driver.maxResultSize", "4g")
        .config("spark.local.dir", local_dir)
    )
    for k, v in kwargs.items():
        builder = builder.config(k, v)
    return builder.getOrCreate()


@contextmanager
def spark_resource(*args, **kwargs):
    """A context manager for a spark session."""
    spark = None
    try:
        spark = get_spark(*args, **kwargs)
        yield spark
    finally:
        if spark is not None:
            spark.stop()

In [4]:
spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 16:28:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/22 16:28:38 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/02/22 16:28:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/22 16:28:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
! gcloud storage ls gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px

gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/_SUCCESS
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/part-00000-5e0e2b9c-4eea-4a49-98fc-fb13c4f30ee5-c000.snappy.parquet
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/part-00001-5e0e2b9c-4eea-4a49-98fc-fb13c4f30ee5-c000.snappy.parquet
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/part-00002-5e0e2b9c-4eea-4a49-98fc-fb13c4f30ee5-c000.snappy.parquet
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/part-00003-5e0e2b9c-4eea-4a49-98fc-fb13c4f30ee5-c000.snappy.parquet
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/part-00004-5e0e2b9c-4eea-4a49-98fc-fb13c4f30ee5-c000.snappy.parquet
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/part-00005-5e0e2b9c-4eea-4a49-98fc-fb13c4f30ee5-c000.snappy.parquet
gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/part-00006-5e0e2b9c-4eea-4a49-98fc-fb13c4f30ee5-c000.snappy.parquet
gs://dsgt-clef-fungiclef-202

In [6]:
gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px"

df = spark.read.option("inferSchema", "true").parquet(gcs_parquet_path)


In [7]:
df.show(5)

24/02/22 16:29:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+-------------+------+-----+----+-----------+--------------------+-------+--------------------+-------+-------------+---------------+-------------+---------------+-------------+---------------+---------+--------------------+---------+----------+---------+-----------+----------+-------------+-----------------+--------------------+-----------------+------------------+---------+----------+--------------------+--------+-------------+---------+
|          image_path|                data|observationID|  year|month| day|countryCode|            locality|taxonID|      scientificName|kingdom|       phylum|          class|        order|         family|        genus|specificEpithet|taxonRank|             species|level0Gid|level0Name|level1Gid| level1Name| level2Gid|   level2Name|    ImageUniqueID|           Substrate|     rightsHolder|          Latitude|Longitude|CoorUncert|             Habitat|class_id|MetaSubstrate|poisonous|
+--------------------+------

In [ ]:
df